## Import libraries, get API client and data

In [1]:
import googlemaps
import pandas as pd

In [2]:
with open(r'..\..\..\Google API key.txt') as f:
    secret = f.readlines()[0]
client = googlemaps.Client(key = secret)

In [6]:
unidades = pd.read_csv(r'..\1. Obtenção de coordenadas geográficas\Lista Final com coordenadas.csv', delimiter=';')
unidades = unidades.drop(columns=['Unnamed: 0'])

In [7]:
unidades.style.set_sticky()

,UBS,ENDEREÇO,REGIONAL DE SAÚDE,coordinates,google maps name
0,UBS 1 ASA SUL,"UBS 1 ASA SUL (CSB08), SGAS QUADRA 612 LOTES S/N, ASA SUL, Brasília-DF, 70200-720",Região Central,"(-15.8326601, -47.909503)",Unidade Básica de Saúde 1 Asa Sul
1,UBS 1 CRUZEIRO,"UBS 1 CRUZEIRO (CSB09), Shces Quadra, 611 - Cruzeiro Novo, Brasília - DF, 70070-710",Região Central,"(-15.800795, -47.942134)",UBS 01 Cruzeiro Novo (GSAP)
2,UBS 1 LAGO NORTE,"UBS 1 LAGO NORTE (CSB10), SHIN QI 3 03, LAGO NORTE, Brasília-DF, 71505-600",Região Central,"(-15.7257082, -47.8739439)",Vacinação COVID-19 - UBS 1 Lago Norte (Centro de Saúde n°10)
3,UBS 1 ASA NORTE,"UBS 1 ASA NORTE (CSB11), SGAN 905, ASA NORTE, Brasília-DF, 70790-054",Região Central,"(-15.7732459, -47.89242780000001)",SAP 01 da Asa Norte
4,UBS 2 ASA NORTE,"UBS 2 ASA NORTE (CSB13), Eqn 208 408, 208 - Asa Norte, Brasília - DF, 70853-450",Região Central,"(-15.7616704, -47.8805846)",CEDOH
5,UBS 3 ASA NORTE,"UBS 3 ASA NORTE, Rua Piau Acampamento Pacheco Fernandes, Área Especial, Vila Planalto, 2, Brasília - DF, 70804-190",Região Central,"(-15.7951717, -47.8481369)",Unidade Básica de Saúde (UBS) 3 - Asa Norte - Vila Planalto
6,UBS 5 ASA NORTE,"UBS 5 ASA NORTE, Vila Weslian Roriz , Área Especial, 1, Granja do Torto, Lago Norte, Brasília - DF, 70636-003",Região Central,"(-15.705292, -47.9059388)",Residência Oficial da Granja do Torto
7,UBS 2 CRUZEIRO,"UBS 2 CRUZEIRO (CSB14), Shce Setor Escolar Lote, 4 - Cruzeiro Velho, Brasília - DF, 70070-710",Região Central,"(-15.7835664, -47.9359858)",Unidade Básica de Saúde (UBS) 2 Cruzeiro
8,UBS 1 VARJÃO,"UBS 1 VARJÃO (CSB16), VILA VARJAO QUADRA 5 05, VILA VARJAO DO TORTO, Brasília-DF, 71555-133",Região Central,"(-15.7109084, -47.876274)",Unidade Básica de Saúde n° 1 Varjão
9,UBS 1 CANDANGOLÂNDIA,"UBS 1 CANDANGOLÂNDIA (CSCAN01) EQ 5 7 AREA ESPECIAL - CANDANGOLANDIA 01, CANDANGOLANDIA, Brasília-DF, 71725- 400",Região Centro - Sul,"(-15.8498126, -47.9513927)",UBS 01 - Candangolândia


In [8]:
unidades.drop([30,31,32]).reset_index()

,index,UBS,ENDEREÇO,REGIONAL DE SAÚDE,coordinates,google maps name
0,0,UBS 1 ASA SUL,"UBS 1 ASA SUL (CSB08), SGAS QUADRA 612 LOTES S...",Região Central,"(-15.8326601, -47.909503)",Unidade Básica de Saúde 1 Asa Sul
1,1,UBS 1 CRUZEIRO,"UBS 1 CRUZEIRO (CSB09), Shces Quadra, 611 - Cr...",Região Central,"(-15.800795, -47.942134)",UBS 01 Cruzeiro Novo (GSAP)
2,2,UBS 1 LAGO NORTE,"UBS 1 LAGO NORTE (CSB10), SHIN QI 3 03, LAGO N...",Região Central,"(-15.7257082, -47.8739439)",Vacinação COVID-19 - UBS 1 Lago Norte (Centro ...
3,3,UBS 1 ASA NORTE,"UBS 1 ASA NORTE (CSB11), SGAN 905, ASA NORTE, ...",Região Central,"(-15.7732459, -47.89242780000001)",SAP 01 da Asa Norte
4,4,UBS 2 ASA NORTE,"UBS 2 ASA NORTE (CSB13), Eqn 208 408, 208 - As...",Região Central,"(-15.7616704, -47.8805846)",CEDOH
...,...,...,...,...,...,...
123,126,UBS 3 SANTA MARIA,"UBS 3 - SANTA MARIA, Qr 100 Conj I Lote, 1 - S...",Região Sul,"(-16.046903804013322, -48.03679898512468)",UBS 03 - Santa Maria
124,127,UBS 5 SANTA MARIA,"UBS 5 - SANTA MARIA, Eq 212 213 Lote E, 1 - Sa...",Região Sul,"(-16.01032982549102, -48.002362702072205)",Unidade Básica de Saúde 5 - UBS 5 DE SANTA MARIA
125,128,UBS 6 SANTA MARIA,"UBS 6 - SANTA MARIA, Qr 202 302 Lote 2, 2 - Sa...",Região Sul,"(-16.040087, -48.0306225)",UBS 06 - Santa Maria
126,129,UBS 7 SANTA MARIA,"UBS 7 - SANTA MARIA, Av Brigadeiro Pinto de Mo...",Região Sul,"(-15.9932117, -47.9879706)",Ubs 7 de Santa Maria


## Get distances

In [9]:
coordenadas_unidades = list(unidades['coordinates'])

In [10]:
centros = pd.read_csv('Centros.csv', delimiter=',')
nomes_centros = list(centros['centros'])

In [11]:
df = pd.DataFrame(columns = nomes_centros)

In [12]:
df.style.set_sticky()

,Hospital Regional da Asa Norte,Hospital Regional da Ceilândia,Hospital Regional do Gama,Hospital Regional do Paranoá,Hospital Regional de Planaltina,Hospital Regional de Taguatinga,Hospital Regional de Sobradinho,UBS 1 - Núcleo Bandeirante


In [9]:
for j in nomes_centros:
    for i in coordenadas_unidades:
        try:
            df.at[i,j] = client.distance_matrix(origins=j,destinations=i.replace('(','').replace(')',''), mode='driving')['rows'][0]['elements'][0]['distance']['value']
        except:
            if j == 'Hospital Regional do Gama':
                try:
                    df.at[i,j] = client.distance_matrix(origins='-16.023317, -48.068846',destinations=i.replace('(','').replace(')',''), mode='driving')['rows'][0]['elements'][0]['distance']['value']
                except:
                    print(i, j)
            else:
                print(i, j)

In [38]:
#df = df.reset_index()
#df.rename(columns = {'Unnamed: 0':'Coordenadas'}, inplace = True)
df = df.drop(columns=['level_0'])

In [34]:
df = df.reset_index()

In [43]:
df['UBS ou ponto de vacinação'] = unidades.drop([30,31,32]).reset_index()['UBS']
df['Nome Google Maps'] = unidades.drop([30,31,32]).reset_index()['google maps name']
df['coordinates	'] = unidades.drop([30,31,32]).reset_index()['coordinates']

In [14]:
df = df.drop(columns=['UBS ou ponto de vacinação', 'Nome Google Maps'])

## Export

In [46]:
df.style.set_sticky()

,Hospital Regional da Asa Norte,Hospital Regional da Ceilândia,Hospital Regional do Gama,Hospital Regional do Paranoá,Hospital Regional de Planaltina,Hospital Regional de Taguatinga,Hospital Regional de Sobradinho,UBS 1 - Núcleo Bandeirante,UBS ou ponto de vacinação,Nome Google Maps,coordinates
0,8997,27385,30544,23025,52194,21739,34686,10494,UBS 1 ASA SUL,Unidade Básica de Saúde 1 Asa Sul,"(-15.8326601, -47.909503)"
1,9577,21300,31480,30722,45078,18944,27570,9379,UBS 1 CRUZEIRO,UBS 01 Cruzeiro Novo (GSAP),"(-15.800795, -47.942134)"
2,10502,30682,44216,19157,37801,29055,20293,22114,UBS 1 LAGO NORTE,Vacinação COVID-19 - UBS 1 Lago Norte (Centro de Saúde n°10),"(-15.7257082, -47.8739439)"
3,2471,27777,41310,22218,41099,26149,23591,19209,UBS 1 ASA NORTE,SAP 01 da Asa Norte,"(-15.7732459, -47.89242780000001)"
4,3637,33149,38972,24150,39998,31521,22490,18923,UBS 2 ASA NORTE,CEDOH,"(-15.7616704, -47.8805846)"
5,8591,32650,38803,17027,46919,31724,29411,18183,UBS 3 ASA NORTE,Unidade Básica de Saúde (UBS) 3 - Asa Norte - Vila Planalto,"(-15.7951717, -47.8481369)"
6,11056,29670,43203,21243,34454,28042,16946,21102,UBS 5 ASA NORTE,Residência Oficial da Granja do Torto,"(-15.705292, -47.9059388)"
7,7677,21074,33096,30496,44852,19446,27344,10995,UBS 2 CRUZEIRO,Unidade Básica de Saúde (UBS) 2 Cruzeiro,"(-15.7835664, -47.9359858)"
8,11543,31723,45256,16051,34441,30096,16933,23155,UBS 1 VARJÃO,Unidade Básica de Saúde n° 1 Varjão,"(-15.7109084, -47.876274)"
9,14712,27102,25577,29713,50880,20628,33372,3393,UBS 1 CANDANGOLÂNDIA,UBS 01 - Candangolândia,"(-15.8498126, -47.9513927)"


In [47]:
df.to_csv('Matriz de distâncias final - 131 unidades.csv')